In [112]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

/Users/yj.noh/Documents/GitHub


In [113]:
data = pd.read_csv("prj-ML-model-LT_OV30/modeling_data.csv", encoding = "cp949")
print(data.head())
print(data.shape)

            dlvry_id  reg_hour  ord_price  actual_dlvry_distance  pick_floor  \
0  20230827_9376QTJN        11      22500                   1274         1.0   
1  20230827_1765MBEB        17      15700                   2809         1.0   
2  20230827_0741UTNW        18      20400                   1961        -1.0   
3  20230827_6306VAVZ        19      16900                   1826         1.0   
4  20230827_5544DEBH        11      21000                   1184         2.0   

  pick_rgn2_nm pick_rgn3_nm pick_category pick_건물용도  dlvry_지상층수  dlvry_지하층수  \
0          서초구          잠원동       CHINESE      업무시설         NaN         NaN   
1          서초구          반포동         SNACK     일반음식점        19.0         5.0   
2          서초구          서초동     ZZIM_TANG       아파트         4.0         1.0   
3          서초구          방배동         ASIAN       아파트         5.0         0.0   
4          서초구          서초동          CAFE       NaN         4.0         1.0   

  dlvry_건물용도 day_of_week  is_holiday    기온  

In [114]:
# 건물 정보 없는 데이터 삭제 
filtered_data = data.dropna(subset=['pick_건물용도', 'dlvry_건물용도'])
print(filtered_data.shape) #44,644 
print(filtered_data['outcome'].value_counts()) # 38,816, 5,828 

(44644, 19)
outcome
0    38816
1     5828
Name: count, dtype: int64


In [115]:
print(filtered_data.isna().sum())

dlvry_id                    0
reg_hour                    0
ord_price                   0
actual_dlvry_distance       0
pick_floor                  0
pick_rgn2_nm                0
pick_rgn3_nm                0
pick_category               0
pick_건물용도                   0
dlvry_지상층수               5726
dlvry_지하층수               5726
dlvry_건물용도                  0
day_of_week                 0
is_holiday                  0
기온                          0
강수량                         0
dlvry_rgn2_nm               0
dlvry_rgn3_nm               0
outcome                     0
dtype: int64


In [116]:
# dlvry_지하층수 음수로
filtered_data['dlvry_지하층수'] = pd.to_numeric(filtered_data['dlvry_지하층수'], errors='coerce')
filtered_data['dlvry_지상층수'] = pd.to_numeric(filtered_data['dlvry_지상층수'], errors='coerce')
filtered_data['pick_floor'] = pd.to_numeric(filtered_data['pick_floor'], errors='coerce')
filtered_data['dlvry_지하층수'] = filtered_data['dlvry_지하층수'].apply(lambda x: -x)
print(filtered_data.head())

            dlvry_id  reg_hour  ord_price  actual_dlvry_distance  pick_floor  \
1  20230827_1765MBEB        17      15700                   2809         1.0   
2  20230827_0741UTNW        18      20400                   1961        -1.0   
3  20230827_6306VAVZ        19      16900                   1826         1.0   
6  20230827_2040JWLP        21      12000                   1148         1.0   
7  20230827_9868PDJF        20      21500                   1105         1.0   

  pick_rgn2_nm pick_rgn3_nm pick_category pick_건물용도  dlvry_지상층수  dlvry_지하층수  \
1          서초구          반포동         SNACK     일반음식점        19.0        -5.0   
2          서초구          서초동     ZZIM_TANG       아파트         4.0        -1.0   
3          서초구          방배동         ASIAN       아파트         5.0        -0.0   
6          서초구          양재동       CHICKEN      업무시설         2.0        -1.0   
7          서초구          서초동       CHICKEN      업무시설        46.0        -3.0   

  dlvry_건물용도 day_of_week  is_holiday    기온  

## 1. 데이터 전처리


## 1-1. pick_건물용도/ dlvry_건물용도 : 개수 축소

In [117]:
print(filtered_data['pick_건물용도'].value_counts())

pick_건물용도
업무시설          14372
근린생활시설         7875
아파트            5367
일반음식점          5206
일반상점           3881
공동주택           3089
교육시설           1604
의료시설           1177
백화점             534
자동차관련시설         471
오피스텔            266
기타시설            210
숙박시설            174
대형판매점           141
위락시설            134
문화시설             87
종교시설             30
체육시설             16
위험물저장및처리시설       10
Name: count, dtype: int64


In [118]:
# 데이터 정리
values_to_replace = ['위락시설', '문화시설', '종교시설', '체육시설','위험물저장및처리시설']
filtered_data['pick_건물용도'] = data['pick_건물용도'].replace(values_to_replace, '기타시설')
print(filtered_data['pick_건물용도'].value_counts())

pick_건물용도
업무시설       14372
근린생활시설      7875
아파트         5367
일반음식점       5206
일반상점        3881
공동주택        3089
교육시설        1604
의료시설        1177
백화점          534
기타시설         487
자동차관련시설      471
오피스텔         266
숙박시설         174
대형판매점        141
Name: count, dtype: int64


In [119]:
print(filtered_data['dlvry_건물용도'].value_counts())

dlvry_건물용도
공동주택          16952
아파트           13910
업무시설           7279
근린생활시설         1559
일반상점           1224
교육시설           1028
의료시설            758
일반음식점           557
숙박시설            459
공공시설            241
종교시설            127
교육연구시설          120
오피스텔            101
자동차관련시설          67
방송통신시설           53
기타시설             39
체육시설             36
백화점              36
위험물저장및처리시설       18
복지시설             16
문화시설             14
위락시설             12
복합몰               7
창고시설              6
수련시설              6
동식물시설             5
단독주택              5
운수시설              3
노유자시설             2
호텔                2
체력시설              1
공공용시설             1
Name: count, dtype: int64


In [120]:
value_counts = filtered_data['dlvry_건물용도'].value_counts()

threshold = 100
filtered_data['dlvry_건물용도'] = filtered_data['dlvry_건물용도'].apply(lambda x: '기타시설' if value_counts[x] <= threshold else x)

filtered_data['dlvry_건물용도'] = filtered_data['dlvry_건물용도'].replace('교육연구시설', '교육시설')
filtered_data['dlvry_건물용도'] = filtered_data['dlvry_건물용도'].replace('공공용시설', '공공시설')

print(filtered_data['dlvry_건물용도'].value_counts())

dlvry_건물용도
공동주택      16952
아파트       13910
업무시설       7279
근린생활시설     1559
일반상점       1224
교육시설       1148
의료시설        758
일반음식점       557
숙박시설        459
기타시설        329
공공시설        241
종교시설        127
오피스텔        101
Name: count, dtype: int64


## 1-2. dlvry_지상층수 /지하층수 : dlvry_건물용도 median 값 가져오기

In [121]:
median_values = filtered_data.dropna().groupby('dlvry_건물용도')[['dlvry_지상층수', 'dlvry_지하층수']].median()
mean_values = filtered_data.dropna().groupby('dlvry_건물용도')[['dlvry_지상층수', 'dlvry_지하층수']].mean()

print("Median Values:")
print(median_values)

print("\nMean Values:")
print(mean_values)


Median Values:
            dlvry_지상층수  dlvry_지하층수
dlvry_건물용도                        
공공시설               3.0        -0.0
공동주택               4.0        -1.0
교육시설               6.0        -1.0
근린생활시설             5.0        -1.0
기타시설               5.0        -2.0
숙박시설              10.0        -1.0
아파트               15.0        -1.0
업무시설              10.0        -3.0
오피스텔              15.0        -7.0
의료시설              10.0        -1.0
일반상점               4.0        -1.0
일반음식점              5.0        -1.0
종교시설               4.0        -3.0

Mean Values:
            dlvry_지상층수  dlvry_지하층수
dlvry_건물용도                        
공공시설          3.066390   -0.203320
공동주택          3.844208   -0.732086
교육시설          6.414397   -1.383268
근린생활시설        5.316228   -1.250160
기타시설          8.412698   -2.501587
숙박시설         10.838428   -2.397380
아파트          18.313515   -1.368009
업무시설         11.697593   -3.120231
오피스텔         15.000000   -6.118812
의료시설          9.507916   -1.928760
일반상점          5.041667   -

In [122]:
# na 값을 중앙값으로 채우기
filtered_data['dlvry_지상층수'].fillna(filtered_data['dlvry_건물용도'].map(median_values['dlvry_지상층수']), inplace=True)
filtered_data['dlvry_지하층수'].fillna(filtered_data['dlvry_건물용도'].map(median_values['dlvry_지하층수']), inplace=True)

print(filtered_data.describe())
print(filtered_data.columns)

           reg_hour      ord_price  actual_dlvry_distance    pick_floor  \
count  44644.000000   44644.000000           44644.000000  44644.000000   
mean      15.780866   27080.946757            1959.281538      0.748006   
std        3.745221   17994.351039            1156.483892      0.924460   
min        9.000000    5000.000000               1.000000     -1.000000   
25%       12.000000   16700.000000            1115.000000      1.000000   
50%       16.000000   22400.000000            1756.000000      1.000000   
75%       19.000000   31400.000000            2514.000000      1.000000   
max       23.000000  483000.000000            8003.000000      8.000000   

         dlvry_지상층수    dlvry_지하층수            기온           강수량       outcome  
count  44644.000000  44644.000000  44644.000000  44644.000000  44644.000000  
mean      10.028380     -1.476481     26.778833      0.609437      0.130544  
std        8.766227      1.608017      2.664765      3.162902      0.336905  
min        0

In [123]:
print(filtered_data.isna().sum())

dlvry_id                 0
reg_hour                 0
ord_price                0
actual_dlvry_distance    0
pick_floor               0
pick_rgn2_nm             0
pick_rgn3_nm             0
pick_category            0
pick_건물용도                0
dlvry_지상층수               0
dlvry_지하층수               0
dlvry_건물용도               0
day_of_week              0
is_holiday               0
기온                       0
강수량                      0
dlvry_rgn2_nm            0
dlvry_rgn3_nm            0
outcome                  0
dtype: int64


## 1-3.  pick_floor, dlvry_지상층수, dlvry_지하층수 

In [124]:
print(filtered_data['pick_floor'].value_counts())


pick_floor
 1.0    31450
-1.0     7548
 2.0     3810
 0.0     1363
 3.0      314
 4.0       85
 8.0       73
 6.0        1
Name: count, dtype: int64


In [125]:
print(filtered_data['dlvry_지하층수'].value_counts())

dlvry_지하층수
-1.0    24736
-0.0     9058
-2.0     3590
-3.0     1876
-4.0     1778
-5.0     1483
-6.0     1233
-7.0      674
-8.0      215
-9.0        1
Name: count, dtype: int64


In [130]:
print(filtered_data['dlvry_지상층수'].value_counts())
print(filtered_data['dlvry_지상층수'].describe())

dlvry_지상층수
4.0     9546
5.0     5935
3.0     4366
15.0    4286
6.0     2892
2.0     1796
12.0    1556
7.0     1408
20.0    1372
10.0    1270
13.0    1182
29.0     872
32.0     782
14.0     722
9.0      713
25.0     599
18.0     551
24.0     526
8.0      522
11.0     466
27.0     413
22.0     412
16.0     329
17.0     269
37.0     267
21.0     262
19.0     248
46.0     184
26.0     177
1.0      171
23.0     166
31.0     114
66.0      52
28.0      43
69.0      40
55.0      30
51.0      26
34.0      22
44.0      20
45.0      16
0.0        9
30.0       8
33.0       4
Name: count, dtype: int64
count    44644.000000
mean        10.028380
std          8.766227
min          0.000000
25%          4.000000
50%          6.000000
75%         15.000000
max         69.000000
Name: dlvry_지상층수, dtype: float64


In [127]:
# def categorize_floor(value):
#     if value in [-1, 0, 1]:
#         return '지하'
#     elif 2 <= value <= 4:
#         return '중층'
#     else:
#         return '고층'

# filtered_data['pick_floor'] = filtered_data['pick_floor'].apply(categorize_pick_floor)
